# Testing Sly Parser for Hebrew Noun Phrases

In [1]:
import json
from tf.app import use
from sly import Lexer, Parser
from sly.lex import Token
import networkx as nx

# load Hebrew data
bhsa = use('bhsa', hoist=globals())

with open('../../results/parsing/slot2pos.json', 'r') as infile:
    slot2pos = {int(k):v for k,v in json.load(infile).items()}

In [2]:
bhsa.displaySetup(condenseType='clause', withNodes=True, extraFeatures='st lex function')

## Get Testing Data

In [3]:
# get Hebrew nodes as tester data

bhsa.indent(reset=True)

queries = [

"""
phrase rela=NA
/where/
    word
/have/
    pdp=subs prs=absent
/-/
    word ls=card
    word pdp=subs ls#card

""",
    
# """
# p:phrase
#     w1:word pdp=subs st=c ls#card
#     <: w2:word pdp=subs ls=card prs#absent

# p =: w1 
# p := w2
# """,
]

samples = []
sample_nodes = []

for q in queries: 
    samples += bhsa.search(q, shallow=True)
    
samples.sort()

sample_nodes = {pn:L.d(pn, 'word') for pn in samples}
    
bhsa.info(f'total samples: {len(samples)}')

  2.57s 1186 results
  2.57s total samples: 1186


In [4]:
list(sample_nodes.values())[:10]

[(77, 78),
 (1992, 1993),
 (2175, 2176, 2177),
 (2245, 2246),
 (2286, 2287),
 (2389, 2390, 2391),
 (2433, 2434, 2435),
 (2587, 2588, 2589, 2590),
 (2876, 2877, 2878),
 (2882, 2883)]

In [5]:
bhsa.show(list(sample_nodes.values())[:40])

<hr>

In [90]:
class CX:
    def __init__(self, slot, pos):
        self.slot = slot
        self.cat = pos
    def __repr__(self):
        return f'CX({self.slot})'
        
class BhsaLexer(Lexer):
    """Wrap a BHSA node in SLY Token object.
        
    This enables us to use our custom tokens rather
    than raw string input.
    """

    # unique parts of speech values harvested from 
    # the parts of speech parser and pasted here
    # see results/parsing/uniquepos.txt
    tokens = {
#         ADJV,
         ADVB,
         ART,
         C,
         CARD,
         CONJ,
#         INRG,
#         INTJ,
#         NEGA,
         NOUN,
#         ORDN,
#         PRDE,
         PREP,
#         PRIN,
#         PROPN,
#         PRPS,
#         QUANT,
         SFX,
#         VERB,
    }
    
    def get_token(self, cx, cat, index):
        """Compile SLY token object."""
        token = Token()
        token.value = cx
        token.type = cat
        token.index = index
        token.lineno = index
        return token
    
    def tokenize(self, slots, slot2pos):
        """Write over the SLY tokenize method to yield custom data.

        Args:
            slots: iterable of integers which correspond with
                slots / word nodes in BHSA
            slot2pos: a dict that maps slots to their parts of speech
        
        Yields:
            generator of SLY Tokens
        """
        i = 0
        for slot in slots:
            
            # configure the word
            pos = slot2pos[slot]
            cx = CX(slot, pos)
            token = self.get_token(cx, pos, i)
            yield token
            i += 1
            
            # split off morphological tokens
            for morph in self.tokenize_morphs(slot):
                yield self.get_token(morph, morph.cat, i)
                i += 1
        
    def tokenize_morphs(self, slot):
        """Tokenize morphological forms."""
        if F.st.v(slot) == 'c' and slot2pos[slot] == 'NOUN':
            yield CX(slot, 'C')
        if F.prs.v(slot) != 'absent':
            yield CX(slot, 'SFX')
        
error_records = []
        
class PhraseParser(Parser):

    """A CFG parser for Biblical Hebrew phrases."""

    tokens = BhsaLexer.tokens

    def error(self, token):
#         bhsa.prettyTuple([token.value.slot], seq=0)
#         ph = L.u(token.value.slot, 'phrase')[0]
#         tokens = list(lexer.tokenize(sample_nodes[ph], slot2pos))
        error_records.append(token)
    
    precedence = [
        ('left', CONJ),
        ('left', CARD, NOUN),
    ]
    
    debugfile = 'parser.out'
    
    # -- FINAL PHRASE --
    @_('np', 'pp','ccard', 
       'num', 'gp_card', 'gp_num')
    def phrase(self, p):
        return p[0]

    # -- noun-based phrases --
    @_('NOUN SFX')
    def np(self, p):
        return p[0].slot
    
    @_('df', 'appo', 'gp')
    def np(self, p):
        return p[0]
    
    # -- definite phrases --
    @_('ART NOUN')
    def df(self, p):
        return [p[0].slot, p[1].slot, 'DF']

    # -- prepositional phrases --
    @_('PREP phrase')
    def pp(self, p):
        return [p[0].slot, p[1], 'PP']
    
    @_('PREP NOUN', 'PREP ADVB')
    def pp(self, p):
        return [p[0].slot, p[1].slot, 'PP']
    
    @_('PREP SFX')
    def pp(self, p):
        return p[0]
    
    # -- genitive phrases --
    @_('NOUN C NOUN')
    def gp(self, p):
        return [p[2].slot, p[0].slot, 'GP']
    
    @_('NOUN C np')
    def gp(self, p):
        return [p[2], p[0].slot, 'GP']
    
    @_('NOUN C num')
    def gp_card(self, p):
        return [p[2], p[0].slot, 'GP_CARD']
    
    @_('NOUN C CARD')
    def gp_num(self, p):
        return [p[0].slot, p[2].slot, 'GP_NUM']
    
    # -- appositional phrases --
    @_('NOUN NOUN')
    def appo(self, p):
        return [p[0].slot, p[1].slot, 'APPO']
    
    @_('df df')
    def appo(self, p):
        return [p[1], p[0], 'APPO']
    
    # -- chained cardinal numbers --
    @_('CARD CARD')
    def ccard(self, p):
        return [p[0].slot, p[1].slot, 'CCARD']
    
    @_('CARD CONJ CARD')
    def ccard(self, p):
        return [p[0].slot, [p[1].slot, p[2].slot, 'CONJ'], 'CCARD']
    
    @_('ccard CONJ ccard')
    def ccard(self, p):
        return [p[0], [p[1].slot, p[2], 'CONJ'], 'CCARD']
    
    @_('CARD CONJ ccard')
    def ccard(self, p):
        return [p[0].slot, [p[1].slot, p[2], 'CONJ'], 'CCARD']
    
    @_('ccard CARD')
    def ccard(self, p):
        return [p[0], p[1].slot, 'CCARD']
    
    # -- cardinal quantifications --
    @_('NOUN CARD', 'CARD NOUN')
    def num(self, p):
        return [p.CARD.slot, p.NOUN.slot, 'NUM']
    
    @_('ccard NOUN')
    def num(self, p):
        return [p[0], p[1].slot, 'NUM']
    
    @_('ccard np')
    def num(self, p):
        return [p[0], p[1], 'NUM']
    
    @_('CARD np')
    def num(self, p):
        return [p[0].slot, p[1], 'NUM']
    
    @_('np CARD')
    def num(self, p):
        return [p[1].slot, p[0], 'NUM']

lexer = BhsaLexer()
parser = PhraseParser()

Parser debugging for PhraseParser written to parser.out


In [77]:
tokens = lexer.tokenize(list(sample_nodes.values())[0], slot2pos)

list(tokens)

[Token(type='NOUN', value=CX(77), lineno=0, index=0),
 Token(type='CARD', value=CX(78), lineno=1, index=1)]

In [78]:
tokens = lexer.tokenize(sample_nodes[676648][:2], slot2pos)

list(tokens)

[Token(type='NOUN', value=CX(40497), lineno=0, index=0),
 Token(type='CARD', value=CX(40498), lineno=1, index=1)]

In [79]:
parser.parse(lexer.tokenize(sample_nodes[676648][:2], slot2pos))

[40498, 40497, 'NUM']

In [89]:
bhsa.prettyTuple([694558], seq=0)

### Large scale tests

In [91]:
parsed = {}
error_records = []

for i, phrase in enumerate(sample_nodes):
    nodeset = sample_nodes[phrase]
    parsing = parser.parse(lexer.tokenize(nodeset, slot2pos))
    parsed[phrase] = parsing

print(len(parsed), 'parsed')
print(len(error_records), 'not parsed')

1186 parsed
61 not parsed


In [96]:
parsed[735746]

[147976, 147977, 'NUM']

In [95]:
parser.parse(lexer.tokenize(L.d(735746, 'word'), slot2pos))

[147976, 147977, 'NUM']

## Display unparsed

In [62]:
for i, token in enumerate(error_records):
    bhsa.prettyTuple([token.value.slot], seq=i)
    ph = L.u(token.value.slot, 'phrase')[0]
    tokens = list(lexer.tokenize(sample_nodes[ph], slot2pos))
    print(tokens)
    print()

[Token(type='CARD', value=CX(7901), lineno=0, index=0), Token(type='NOUN', value=CX(7902), lineno=1, index=1), Token(type='NOUN', value=CX(7903), lineno=2, index=2), Token(type='NOUN', value=CX(7904), lineno=3, index=3)]



[Token(type='NOUN', value=CX(40497), lineno=0, index=0), Token(type='CARD', value=CX(40498), lineno=1, index=1), Token(type='NOUN', value=CX(40499), lineno=2, index=2)]



[Token(type='NOUN', value=CX(44015), lineno=0, index=0), Token(type='C', value=CX(44015), lineno=1, index=1), Token(type='NOUN', value=CX(44016), lineno=2, index=2), Token(type='CARD', value=CX(44017), lineno=3, index=3), Token(type='NOUN', value=CX(44018), lineno=4, index=4), Token(type='NOUN', value=CX(44019), lineno=5, index=5)]



[Token(type='NOUN', value=CX(45368), lineno=0, index=0), Token(type='NOUN', value=CX(45369), lineno=1, index=1), Token(type='C', value=CX(45369), lineno=2, index=2), Token(type='NOUN', value=CX(45370), lineno=3, index=3), Token(type='CARD', value=CX(45371), lineno=4, index=4)]



[Token(type='CARD', value=CX(49775), lineno=0, index=0), Token(type='NOUN', value=CX(49776), lineno=1, index=1), Token(type='CARD', value=CX(49777), lineno=2, index=2), Token(type='NOUN', value=CX(49778), lineno=3, index=3)]



...

[Token(type='NOUN', value=CX(412553), lineno=0, index=0), Token(type='CARD', value=CX(412554), lineno=1, index=1), Token(type='CARD', value=CX(412555), lineno=2, index=2)]



[Token(type='NOUN', value=CX(413447), lineno=0, index=0), Token(type='CARD', value=CX(413448), lineno=1, index=1), Token(type='CARD', value=CX(413449), lineno=2, index=2)]



[Token(type='NOUN', value=CX(414843), lineno=0, index=0), Token(type='C', value=CX(414843), lineno=1, index=1), Token(type='NOUN', value=CX(414844), lineno=2, index=2), Token(type='CARD', value=CX(414845), lineno=3, index=3), Token(type='CARD', value=CX(414846), lineno=4, index=4), Token(type='CARD', value=CX(414847), lineno=5, index=5)]



[Token(type='NOUN', value=CX(421720), lineno=0, index=0), Token(type='CARD', value=CX(421721), lineno=1, index=1), Token(type='NOUN', value=CX(421722), lineno=2, index=2), Token(type='CARD', value=CX(421723), lineno=3, index=3), Token(type='NOUN', value=CX(421724), lineno=4, index=4), Token(type='CARD', value=CX(421725), lineno=5, index=5)]



[Token(type='NOUN', value=CX(425532), lineno=0, index=0), Token(type='CARD', value=CX(425533), lineno=1, index=1), Token(type='CARD', value=CX(425534), lineno=2, index=2)]



## Display Parsed

In [63]:
# bhsa.prettyTuple(L.u(59835, 'phrase'), seq=0, extraFeatures='rela')

In [64]:
def get_nodes(pgraph):
    if type(pgraph) == int:
        yield pgraph
    else:
        for node in pgraph:
            if type(node) == int:
                yield node
            elif type(node) == list:
                yield from get_nodes(node)

def print_relas(pgraph):
    """Print phrase relas embedded in a phrase."""

    snode, tnode, mod = pgraph
    if tnode is not None:
        print(snode, f'-{mod}>', list(get_nodes(tnode)))
    else:
        print(snode, mod)
    if type(snode) == list:
        print_relas(snode)
    elif type(tnode) == list:
        print_relas(tnode)

In [65]:
def show_phrases(parsed_phrases):
    for i,parse in enumerate(parsed_phrases):
        nodes = tuple(get_nodes(parse))
        bhsa.prettyTuple(nodes, seq=i, condenseType='clause', withNodes=True)
        print_relas(parse)
        print()

In [66]:
import random

In [68]:
shuff_parsed = [p for p in parsed.values()]
random.shuffle(shuff_parsed)

show_phrases(shuff_parsed[:20])

113875 -NUM> [113876]



372146 -NUM> [372145]



286853 -NUM> [286854]



213287 -NUM> [213288]



75046 -GP_CARD> [75047, 75048, 75049]
[75047, 75048, 'CCARD'] -NUM> [75049]
75047 -CCARD> [75048]



...

264852 -NUM> [264853]



370201 -NUM> [370200]



391448 -NUM> [391449]



147976 -NUM> [147977]



TypeError: 'NoneType' object is not iterable

# Graph Testing

In [53]:
graph = nx.DiGraph()
graph2 = nx.DiGraph()
graph.add_edge((1,), (3, 4), mod='DEF')
graph2.add_edge((5,), (6, 7), mod='DEF')

# make supergraph with conjunctive connection
graph3 = nx.DiGraph()
graph3.add_edge(graph, graph2, mod='CONJ')

In [92]:
G = nx.DiGraph()

G.add_node(1)